In [1]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import *
import json
import requests
from pprint import pprint
from config import driver, username, password, host, port, database
from sqlalchemy import create_engine
from time import ctime

In [2]:
pd.options.display.max_columns = 30

In [3]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [4]:
contentChoiceDF = pd.read_sql_table('contentChoice', connection)
contentChoiceDF

,id,locationName,shortName,city,latitude,longitude,homePage
0,1,"University of California, Los Angeles",UCLA,Los Angeles,34.079560,-118.444940,Y
1,2,"University of California, Berkeley",UCB,Berkeley,37.871853,-122.258423,N
2,3,"University of California, San Diego",UCSD,La Jolla,32.878880,-117.235930,N
3,4,"University of California, Santa Barbara",UCSB,Santa Barbara,34.413963,-119.848946,N
4,5,"University of California, Irvine",UCI,Irvine,33.640990,-117.844370,N


In [5]:
city = "Los Angeles"
lat = 34.0522
lon = -118.2437

In [6]:
## Get initial endpoint for city
# Assign initial api url to a variable
lat_lon_url = f"https://api.weather.gov/points/{lat},{lon}"

# API call to retrieve enpoints for location of lat and lon
response = requests.get(lat_lon_url).json()

In [7]:
pprint(response)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
              {'@version': '1.1',
               '@vocab': 'https://api.weather.gov/ontology#',
               'bearing': {'@type': 's:QuantitativeValue'},
               'city': 's:addressLocality',
               'county': {'@type': '@id'},
               'distance': {'@id': 's:Distance',
                            '@type': 's:QuantitativeValue'},
               'forecastGridData': {'@type': '@id'},
               'forecastOffice': {'@type': '@id'},
               'geo': 'http://www.opengis.net/ont/geosparql#',
               'geometry': {'@id': 's:GeoCoordinates',
                            '@type': 'geo:wktLiteral'},
               'publicZone': {'@type': '@id'},
               's': 'https://schema.org/',
               'state': 's:addressRegion',
               'unit': 'http://codes.wmo.int/common/unit/',
               'unitCode': {'@id': 's:unitCode', '@type': '@id'},
               'value': {'@id': 's:valu

In [8]:
# Specify the URL
# Major cities in California
weekly_url = response["properties"]["forecast"]
hourly_url = response["properties"]["forecastHourly"]
tz =  response["properties"]["timeZone"]

In [9]:
# Make request and store response
weekly_response = requests.get(weekly_url).json()
weekly_response

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-118.2618399, 34.062663],
    [-118.25719459999999, 34.0404501],
    [-118.23047399999999, 34.0442838],
    [-118.23511409999999, 34.0664973],
    [-118.2618399, 34.062663]]]},
 'properties': {'updated': '2021-10-20T03:38:21+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2021-10-20T04:09:01+00:00',
  'updateTime': '2021-10-20T03:38:21+00:00',
  'validTimes': '2021-10-19T21:00:00+00:00/P7DT16H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 114.9096},
  'periods': [{'number': 1,
    'name': 'Tonight',
    'startTime': '2021-10-19T21:00:00-07:00',
    'endTime': '2021-10-20T06:00:

In [10]:
# # Find generatedAt string; convert to datetime object
# call_datetime_str = weekly_response["properties"]["generatedAt"]
# call_datetime = parse(call_datetime_str)

In [11]:
# Find period forecasts; assign to variable
period_forecasts = weekly_response["properties"]["periods"]

In [12]:
weekly_forecast = []
for i, period_forecast in enumerate(period_forecasts):
    # Isolate date and time from StartTime and EndTime; convert to datetime object
    start_date = datetime.strptime(period_forecasts[i]["startTime"].split("T")[0],"%Y-%m-%d").date()
    start_time = datetime.strptime(period_forecasts[i]["startTime"].split("T")[1],"%H:%M:%S%z").time()
    end_date = datetime.strptime(period_forecasts[i]["endTime"].split("T")[0],"%Y-%m-%d").date()
    end_time = datetime.strptime(period_forecasts[i]["endTime"].split("T")[1],"%H:%M:%S%z").time()
    
    # Get min, max wind speeds; get wind speed units
    wind_speed = period_forecasts[i]["windSpeed"].split(" ")
    if len(wind_speed) == 2:
        min_wind_speed = float(wind_speed[0])
        max_wind_speed = np.nan
        wind_speed_unit = wind_speed[1]
    elif len(wind_speed) == 4:
        min_wind_speed = float(wind_speed[0])
        max_wind_speed = float(wind_speed[2])
        wind_speed_unit = wind_speed[3]

    # Append dates and times to period_forecast dictionary
    period_forecast["city"] = city
    period_forecast["startDate"] = start_date
    period_forecast["start_time"] = start_time
    period_forecast["endDate"] = end_date
    period_forecast["end_time"] = end_time
    period_forecast["minWindSpeed"] = min_wind_speed
    period_forecast["maxWindSpeed"] = max_wind_speed
    period_forecast["windSpeedUnit"] = wind_speed_unit
    period_forecast["latitude"] = lat
    period_forecast["longitude"] = lon
    period_forecast["retrievalDateTime"] = ctime()
    
    # Append period_forcast to weekly_forecast list
    weekly_forecast.append(period_forecast)

# Create dataframe from "weekly_forecast" list
weekly_forecast_raw_df = pd.DataFrame(weekly_forecast)

In [13]:
# Copy weekly_forecast_raw_df to working dataframe
weekly_forecast_working_df = weekly_forecast_raw_df

# Convert startTime from string to datetime object
weekly_forecast_working_df["startTime"] = pd.to_datetime(weekly_forecast_working_df["startTime"])

# Convert endtTime from string to datetime object
weekly_forecast_working_df["endTime"] = pd.to_datetime(weekly_forecast_working_df["endTime"])

# Convert isDayTime from string to boolean
weekly_forecast_working_df.replace({"isDaytime": {"True": True, "False": False}}, inplace=True)

# Reasign column names
weekly_forecast_working_df.rename(columns={"startTime":"startDateTime", "endTime":"endDateTime", "start_time":"startTime", "end_time":"endTime", "number":"responseNumber", "name":"responseName"}, inplace=True)

# Rearange columns of dataframe
weekly_forecast_working_df = weekly_forecast_working_df[['responseNumber', 'responseName', 'city', 'latitude', 'longitude','startDateTime', 'startDate', 'startTime', 'endDateTime', 'endDate', 'endTime', 'isDaytime', 'temperature', 'temperatureUnit', 'temperatureTrend', 'windSpeed','minWindSpeed', 'maxWindSpeed', 'windSpeedUnit', 'windDirection', 'icon', 'shortForecast', 'detailedForecast', 'retrievalDateTime']]

In [14]:
# Copy working dataframe to final dataframe
weekly_forecast_df = weekly_forecast_working_df

In [15]:
# Make hourly forecast request and store response
hourly_response = requests.get(hourly_url).json()

In [16]:
# # Find generatedAt string; convert to datetime object
# request_datetime_str = hourly_response["properties"]["generatedAt"]
# request_datetime = parse(call_datetime_str)

In [17]:
# Find hour forecasts; assign to variable
hour_forecasts = hourly_response["properties"]["periods"]
hour_forecasts

[{'number': 1,
  'name': '',
  'startTime': '2021-10-20T01:00:00-07:00',
  'endTime': '2021-10-20T02:00:00-07:00',
  'isDaytime': False,
  'temperature': 54,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'windSpeed': '0 mph',
  'windDirection': 'S',
  'icon': 'https://api.weather.gov/icons/land/night/sct?size=small',
  'shortForecast': 'Partly Cloudy',
  'detailedForecast': ''},
 {'number': 2,
  'name': '',
  'startTime': '2021-10-20T02:00:00-07:00',
  'endTime': '2021-10-20T03:00:00-07:00',
  'isDaytime': False,
  'temperature': 53,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'windSpeed': '0 mph',
  'windDirection': 'SE',
  'icon': 'https://api.weather.gov/icons/land/night/sct?size=small',
  'shortForecast': 'Partly Cloudy',
  'detailedForecast': ''},
 {'number': 3,
  'name': '',
  'startTime': '2021-10-20T03:00:00-07:00',
  'endTime': '2021-10-20T04:00:00-07:00',
  'isDaytime': False,
  'temperature': 52,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  '

In [18]:
hourly_forecast = []
for i, hour_forecast in enumerate(hour_forecasts):
    # Isolate date and time from StartTime and EndTime; convert to datetime object
    start_date = datetime.strptime(hour_forecasts[i]["startTime"].split("T")[0],"%Y-%m-%d").date()
    start_time = datetime.strptime(hour_forecasts[i]["startTime"].split("T")[1],"%H:%M:%S%z").time()
    end_date = datetime.strptime(hour_forecasts[i]["endTime"].split("T")[0],"%Y-%m-%d").date()
    end_time = datetime.strptime(hour_forecasts[i]["endTime"].split("T")[1],"%H:%M:%S%z").time()
    
    # Get hour wind speed; get wind speed units
    wind_speed = hour_forecasts[i]["windSpeed"].split(" ")
    hour_wind_speed = float(wind_speed[0])
    wind_speed_unit = wind_speed[1]

    # Append dates and times to hour_forecast dictionary
    hour_forecast["city"] = city
    hour_forecast["startDate"] = start_date
    hour_forecast["start_time"] = start_time
    hour_forecast["endDate"] = end_date
    hour_forecast["end_time"] = end_time
    hour_forecast["hourWindSpeed"] = hour_wind_speed
    hour_forecast["windSpeedUnit"] = wind_speed_unit
    hour_forecast["latitude"] = lat
    hour_forecast["longitude"] = lon
    hour_forecast["retrievalDateTime"] = ctime()
    
    # Append period_forcast to hourly_forecast list
    hourly_forecast.append(hour_forecast)

# Create dataframe from "hourly_forecast" list
hourly_forecast_raw_df = pd.DataFrame(hourly_forecast)

In [19]:
# Copy hourly_forecast_raw_df to working dataframe
hourly_forecast_working_df = hourly_forecast_raw_df

# Convert startTime from string to datetime object
hourly_forecast_working_df["startTime"] = pd.to_datetime(hourly_forecast_working_df["startTime"])

# Convert endtTime from string to datetime object
hourly_forecast_working_df["endTime"] = pd.to_datetime(hourly_forecast_working_df["endTime"])

# Convert isDayTime from string to boolean
hourly_forecast_working_df.replace({"isDaytime": {"True": True, "False": False}}, inplace=True)

# Reasign column names
hourly_forecast_working_df.rename(columns={"startTime":"startDateTime", "endTime":"endDateTime", "start_time":"startTime", "end_time":"endTime", "number":"responseNumber"}, inplace=True)

# Drop empty columns "name" and "detailedForecast"
hourly_forecast_working_df.drop(columns=["name", "detailedForecast"], inplace=True)

# Rearange columns of dataframe
hourly_forecast_working_df = hourly_forecast_working_df[['responseNumber', 'city', 'latitude', 'longitude','startDateTime', 'startDate', 'startTime', 'endDateTime', 'endDate', 'endTime', 'isDaytime', 'temperature', 'temperatureUnit', 'temperatureTrend', 'windSpeed','hourWindSpeed', 'windSpeedUnit', 'windDirection', 'icon', 'shortForecast', 'retrievalDateTime']]

In [20]:
# Copy working dataframe to final dataframe
hourly_forecast_df = hourly_forecast_working_df

In [21]:
hourly_forecast_df

,responseNumber,city,latitude,longitude,startDateTime,startDate,startTime,endDateTime,endDate,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,hourWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,retrievalDateTime
0,1,Los Angeles,34.0522,-118.2437,2021-10-20 01:00:00-07:00,2021-10-20,01:00:00,2021-10-20 02:00:00-07:00,2021-10-20,02:00:00,False,54,F,None,0 mph,0.0,mph,S,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Wed Oct 20 01:05:17 2021
1,2,Los Angeles,34.0522,-118.2437,2021-10-20 02:00:00-07:00,2021-10-20,02:00:00,2021-10-20 03:00:00-07:00,2021-10-20,03:00:00,False,53,F,None,0 mph,0.0,mph,SE,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Wed Oct 20 01:05:17 2021
2,3,Los Angeles,34.0522,-118.2437,2021-10-20 03:00:00-07:00,2021-10-20,03:00:00,2021-10-20 04:00:00-07:00,2021-10-20,04:00:00,False,52,F,None,0 mph,0.0,mph,SE,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Wed Oct 20 01:05:17 2021
3,4,Los Angeles,34.0522,-118.2437,2021-10-20 04:00:00-07:00,2021-10-20,04:00:00,2021-10-20 05:00:00-07:00,2021-10-20,05:00:00,False,52,F,None,0 mph,0.0,mph,SE,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Wed Oct 20 01:05:17 2021
4,5,Los Angeles,34.0522,-118.2437,2021-10-20 05:00:00-07:00,2021-10-20,05:00:00,2021-10-20 06:00:00-07:00,2021-10-20,06:00:00,False,52,F,None,0 mph,0.0,mph,WSW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Wed Oct 20 01:05:17 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,152,Los Angeles,34.0522,-118.2437,2021-10-26 08:00:00-07:00,2021-10-26,08:00:00,2021-10-26 09:00:00-07:00,2021-10-26,09:00:00,True,53,F,None,5 mph,5.0,mph,N,https://api.weather.gov/icons/land/day/rain?si...,Slight Chance Light Rain,Wed Oct 20 01:05:17 2021
152,153,Los Angeles,34.0522,-118.2437,2021-10-26 09:00:00-07:00,2021-10-26,09:00:00,2021-10-26 10:00:00-07:00,2021-10-26,10:00:00,True,57,F,None,5 mph,5.0,mph,N,https://api.weather.gov/icons/land/day/rain?si...,Slight Chance Light Rain,Wed Oct 20 01:05:17 2021
153,154,Los Angeles,34.0522,-118.2437,2021-10-26 10:00:00-07:00,2021-10-26,10:00:00,2021-10-26 11:00:00-07:00,2021-10-26,11:00:00,True,60,F,None,5 mph,5.0,mph,N,https://api.weather.gov/icons/land/day/rain?si...,Slight Chance Light Rain,Wed Oct 20 01:05:17 2021
154,155,Los Angeles,34.0522,-118.2437,2021-10-26 11:00:00-07:00,2021-10-26,11:00:00,2021-10-26 12:00:00-07:00,2021-10-26,12:00:00,True,63,F,None,5 mph,5.0,mph,W,https://api.weather.gov/icons/land/day/few?siz...,Sunny,Wed Oct 20 01:05:17 2021


In [22]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [23]:
weekly_forecast_df.to_sql('dailyForecastTB',connection, if_exists='append', index=False)

In [24]:
hourly_forecast_df.to_sql('hourlyForecastTB',connection, if_exists='append', index=False)

In [25]:
DailyForecastTblDF = pd.read_sql_table('dailyForecastTB', connection)
DailyForecastTblDF

,id,responseNumber,responseName,city,latitude,longitude,startDateTime,startDate,startTime,endDateTime,endDate,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,minWindSpeed,maxWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,detailedForecast,retrievalDateTime
0,72,1,This Afternoon,Los Angeles,34.0522,-118.2437,2021-10-19 22:00:00,2021-10-19,15:00:00,2021-10-20 01:00:00,2021-10-19,18:00:00,True,71.0,F,None,10 mph,10.0,NaN,mph,SW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 71. Southwest wind aro...",2021-10-19 22:09:19
1,73,2,Tonight,Los Angeles,34.0522,-118.2437,2021-10-20 01:00:00,2021-10-19,18:00:00,2021-10-20 13:00:00,2021-10-20,06:00:00,False,51.0,F,None,0 to 10 mph,0.0,10.0,mph,SSW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 51. South sou...",2021-10-19 22:09:19
2,74,3,Wednesday,Los Angeles,34.0522,-118.2437,2021-10-20 13:00:00,2021-10-20,06:00:00,2021-10-21 01:00:00,2021-10-20,18:00:00,True,70.0,F,None,0 to 10 mph,0.0,10.0,mph,SW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 70. Southwest w...",2021-10-19 22:09:19
3,75,4,Wednesday Night,Los Angeles,34.0522,-118.2437,2021-10-21 01:00:00,2021-10-20,18:00:00,2021-10-21 13:00:00,2021-10-21,06:00:00,False,53.0,F,None,0 to 10 mph,0.0,10.0,mph,SE,https://api.weather.gov/icons/land/night/fog?s...,Patchy Fog,"Patchy fog after 11pm. Mostly cloudy, with a l...",2021-10-19 22:09:19
4,76,5,Thursday,Los Angeles,34.0522,-118.2437,2021-10-21 13:00:00,2021-10-21,06:00:00,2021-10-22 01:00:00,2021-10-21,18:00:00,True,76.0,F,None,0 to 10 mph,0.0,10.0,mph,NNE,https://api.weather.gov/icons/land/day/fog/bkn...,Patchy Fog then Partly Sunny,"Patchy fog before 11am. Partly sunny, with a h...",2021-10-19 22:09:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,235,10,Sunday,Los Angeles,34.0522,-118.2437,2021-10-24 13:00:00,2021-10-24,06:00:00,2021-10-25 01:00:00,2021-10-24,18:00:00,True,68.0,F,None,5 to 15 mph,5.0,15.0,mph,SE,https://api.weather.gov/icons/land/day/bkn/rai...,Partly Sunny then Slight Chance Light Rain,A slight chance of rain after 5pm. Partly sunn...,2021-10-20 01:05:16
178,236,11,Sunday Night,Los Angeles,34.0522,-118.2437,2021-10-25 01:00:00,2021-10-24,18:00:00,2021-10-25 13:00:00,2021-10-25,06:00:00,False,56.0,F,None,10 to 15 mph,10.0,15.0,mph,SSW,https://api.weather.gov/icons/land/night/rain?...,Chance Light Rain,"A chance of rain. Mostly cloudy, with a low ar...",2021-10-20 01:05:16
179,237,12,Monday,Los Angeles,34.0522,-118.2437,2021-10-25 13:00:00,2021-10-25,06:00:00,2021-10-26 01:00:00,2021-10-25,18:00:00,True,64.0,F,None,15 mph,15.0,NaN,mph,SSW,https://api.weather.gov/icons/land/day/rain?si...,Light Rain Likely,"Rain likely. Partly sunny, with a high near 64.",2021-10-20 01:05:16
180,238,13,Monday Night,Los Angeles,34.0522,-118.2437,2021-10-26 01:00:00,2021-10-25,18:00:00,2021-10-26 13:00:00,2021-10-26,06:00:00,False,50.0,F,None,5 to 15 mph,5.0,15.0,mph,NW,https://api.weather.gov/icons/land/night/rain?...,Chance Light Rain,"A chance of rain. Mostly cloudy, with a low ar...",2021-10-20 01:05:16


In [26]:
HourlyForecastTblDF = pd.read_sql_table('hourlyForecastTB', connection)
HourlyForecastTblDF

,id,responseNumber,city,latitude,longitude,startDateTime,startDate,startTime,endDateTime,endDate,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,hourWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,retrievalDateTime
0,1,1,Los Angeles,34.0522,-118.2437,2021-10-20 05:00:00,2021-10-19,22:00:00,2021-10-20 06:00:00,2021-10-19,23:00:00,False,56.0,F,None,5 mph,5.0,mph,WSW,https://api.weather.gov/icons/land/night/skc?s...,Clear,2021-10-19 22:34:39
1,2,2,Los Angeles,34.0522,-118.2437,2021-10-20 06:00:00,2021-10-19,23:00:00,2021-10-20 07:00:00,2021-10-20,00:00:00,False,56.0,F,None,0 mph,0.0,mph,S,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,2021-10-19 22:34:39
2,3,3,Los Angeles,34.0522,-118.2437,2021-10-20 07:00:00,2021-10-20,00:00:00,2021-10-20 08:00:00,2021-10-20,01:00:00,False,55.0,F,None,0 mph,0.0,mph,S,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,2021-10-19 22:34:39
3,4,4,Los Angeles,34.0522,-118.2437,2021-10-20 08:00:00,2021-10-20,01:00:00,2021-10-20 09:00:00,2021-10-20,02:00:00,False,54.0,F,None,0 mph,0.0,mph,S,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,2021-10-19 22:34:39
4,5,5,Los Angeles,34.0522,-118.2437,2021-10-20 09:00:00,2021-10-20,02:00:00,2021-10-20 10:00:00,2021-10-20,03:00:00,False,53.0,F,None,0 mph,0.0,mph,SE,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,2021-10-19 22:34:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,308,152,Los Angeles,34.0522,-118.2437,2021-10-26 15:00:00,2021-10-26,08:00:00,2021-10-26 16:00:00,2021-10-26,09:00:00,True,53.0,F,None,5 mph,5.0,mph,N,https://api.weather.gov/icons/land/day/rain?si...,Slight Chance Light Rain,2021-10-20 01:05:17
308,309,153,Los Angeles,34.0522,-118.2437,2021-10-26 16:00:00,2021-10-26,09:00:00,2021-10-26 17:00:00,2021-10-26,10:00:00,True,57.0,F,None,5 mph,5.0,mph,N,https://api.weather.gov/icons/land/day/rain?si...,Slight Chance Light Rain,2021-10-20 01:05:17
309,310,154,Los Angeles,34.0522,-118.2437,2021-10-26 17:00:00,2021-10-26,10:00:00,2021-10-26 18:00:00,2021-10-26,11:00:00,True,60.0,F,None,5 mph,5.0,mph,N,https://api.weather.gov/icons/land/day/rain?si...,Slight Chance Light Rain,2021-10-20 01:05:17
310,311,155,Los Angeles,34.0522,-118.2437,2021-10-26 18:00:00,2021-10-26,11:00:00,2021-10-26 19:00:00,2021-10-26,12:00:00,True,63.0,F,None,5 mph,5.0,mph,W,https://api.weather.gov/icons/land/day/few?siz...,Sunny,2021-10-20 01:05:17


In [27]:
session.close()

NameError: name 'session' is not defined

NameError: name 'script' is not defined

SyntaxError: EOL while scanning string literal (<ipython-input-30-c2f927f8f54c>, line 1)